In [1]:
import os
import glob
import pandas as pd
import sqlite3
from sqlite3 import Error
from datetime import datetime as dt

In [2]:
#define a couple of relevant paths to get to data/where to save it
PATH=r'C:\Users\andyk\OneDrive\Documents\Python_Scripts\ATP' 
atp_path=r'C:\Users\andyk\OneDrive\Documents\Python_Scripts\ATP\tennis_atp-master\tennis_atp-master'
pbp_path=r'C:\Users\andyk\OneDrive\Documents\Python_Scripts\ATP\tennis_pointbypoint-master\tennis_pointbypoint-master'

In [3]:
#read in all of the files in the directory with certain formats
#lists of path names
all_matches=glob.glob(os.path.join(atp_path, "atp_matches_[0-9]*.csv"))
all_rankings=glob.glob(os.path.join(atp_path, "atp_rankings_*.csv"))

In [10]:
#turn all the dfs of the same type (different years) into one big df
def combine_dfs(globs):
    
    glob_list = []
    for f in globs:
        df = pd.read_csv(f, sep=',')
        df['file'] = f.split('/')[-1]
        glob_list.append(df)

    glob_df = pd.concat(glob_list, ignore_index=True, sort=True)
    return glob_df

matches_df=combine_dfs(all_matches)
rankings_df=combine_dfs(all_rankings)

print(matches_df["tourney_date"])
matches_df["tournament_date"]=matches_df["tourney_date"].apply(lambda x: dt.strptime(str(x), "%Y%m%d"))
matches_df["Year"]=matches_df.tournament_date.dt.year
#date_series=matches_df["tourney_date"].apply(lambda x: str(x))
#tourney_name_joined=matches_df["tourney_name"].str.replace(" ", "_")
matches_df["players"]=matches_df[["winner_name", "loser_name"]].values.tolist()
matches_df["sorted_players"]=matches_df["players"].apply(lambda x: sorted(x))
#matches_df["match_id"]=
#matches_df["match_id"]

0         19680708
1         19680708
2         19680708
3         19680708
4         19680708
            ...   
176946    20200306
176947    20200306
176948    20200306
176949    20200306
176950    20200306
Name: tourney_date, Length: 176951, dtype: int64


In [9]:
#save as csvs
matches_df.to_csv(os.path.join(PATH, "all_matches.csv"), index=False)
rankings_df.to_csv(os.path.join(PATH, "rankings.csv"))

In [19]:
#save in sqlite database
sql_path=r'C:\Users\andyk\OneDrive\Documents\Python_Scripts\SQL\ATP.sqlite'

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

def create_table_pandas(df, name, connection):
    df_headers=list(df.columns.values)
    df_str="("+",".join(df_headers)+")"
    sql_name="CREATE TABLE %s" %(name)
    sql=sql_name+df_str
    try:
        cursor=connection.cursor()
        cursor.execute(sql)
        print("Table: %s created successfully" %(name))
    except Error as e:
        print(f"The error '{e}' occurred")

connection=create_connection(sql_path)


Connection to MySQL DB successful


In [20]:
#create_table_pandas(matches_df, "Matches", connection=connection)
#create_table_pandas(rankings_df, "Rankings", connection=connection)
matches_df.to_sql("Matches", con=connection, if_exists="replace", index=False)
rankings_df.to_sql("Rankings", con=connection, if_exists="replace", index=False)